# Conventions

0. Use multiply on *left* convention: T(x) = A @ x ( column vectors)
1. The Z axis is going out of the camera, meaning clockwise == positive
2. If the object moves anit-clockwise, the camera is moving clockwise

In [ ]:

%matplotlib notebook
import getpass 
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

username = getpass. getuser() 
if username == "argusm":
    recording_dir = "/home/argusm/tmp/control_test"
else:
    recording_dir = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/control_test"
    
episode_num = 1
recording_fn = "{}/episode_{}.npz".format(recording_dir, episode_num)
mask_recording_fn = "{}/episode_{}_mask.npz".format(recording_dir, episode_num)
keep_fn = "{}/episode_{}_keep.npz".format(recording_dir, episode_num)

recording_dict = np.load(recording_fn)
image_recording = recording_dict["rgb_unscaled"]
depth_recording = recording_dict["depth_imgs"]
state_recording = recording_dict["robot_state_full"]
ee_positions = state_recording[:,:3]
actions = recording_dict["actions"]

num_frames = image_recording.shape[0]-1
masks = np.load(mask_recording_fn)["mask"]
np.savez(keep_fn, keep=np.ones(num_frames,dtype=bool))

print(list(recording_dict.keys()))
print(mask_recording_fn)
print(state_recording.shape)

In [ ]:
# plot masked video
plot_masked_video = True
if plot_masked_video:
    fig, ax = plt.subplots(1,1)
    line = ax.imshow(masks[25])
    ax.set_axis_off()

    def update(i):
        image = image_recording[i].copy()
        mask = masks[i]
        image[np.logical_not(mask)] = 255,255,255
        line.set_data(image)
        fig.canvas.draw_idle()

    slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                                 layout=Layout(width='70%'))

    interact(update, i=slider_i2)

In [ ]:
from gym_grasping.flow_control.servoing_module import ServoingModule
servo_module = ServoingModule(recording_dir, episode_num=episode_num, plot=False)

In [ ]:
start_frame = 136
target_frame = 137
#start_frame = 170
#target_frame = 180
servo_module.set_base_frame(target_frame)

In [ ]:
plot_two_frames = True
if plot_two_frames:
    image_start = image_recording[start_frame].copy()
#     image_start[np.logical_not(masks[start_frame])] = 255
    image_target = image_recording[target_frame].copy()
#     image_target[np.logical_not(masks[target_frame])] = 255

    fig, ax = plt.subplots(1,1)
    plt_handle = ax.imshow(image_start)
    ax.set_axis_off()
    def update(i):
        if i == 0:
            plt_handle.set_data(image_start)
        if i == 1:
            plt_handle.set_data(image_target)
        fig.canvas.draw_idle()
    slider_i2 = widgets.IntSlider(min=0,max=1, step=1,value=0,
                                 layout=Layout(width='70%'))
    interact(update, i=slider_i2)
    plt.show()

# Tips 
1. The Z axis is going out of the camera, meaning clockwise == positive
2. If the object moves anit-clockwise, the camera is moving clockwise


In [ ]:
from math import pi
from scipy.spatial.transform import Rotation as R
np.set_printoptions(suppress=True)

# Points_tcp = T_tcp_cam @ Points_cam 
T_tcp_cam = np.array([
 [ 0.99987185, -0.00306941, -0.01571176,  0.00169436],
 [-0.00515523,  0.86743151, -0.49752989,  0.11860651],
 [ 0.015156,    0.49754713,  0.86730453, -0.18967231],
 [ 0.,          0.,          0.,          1.        ]])

def kuka2dcm(data):
    xyz = data[0:3]
    r = R.from_euler('ZYX', data[3:6], degrees=False)
    dcm = np.eye(4)
    dcm[:3,3] = xyz
    dcm[:3,:3] = r.as_matrix()
    return dcm

def dcm2pretty(dcm):
    trn = dcm[:3,3]
    rot = R.from_dcm(dcm[:3,:3]).as_euler('xyz')
    return np.concatenate((trn,rot)).round(3)

# state has the structure: x,y,z,rot_z,rot_y,rot_x, joint[0-6], desired tcp pos x,y,z, rot_z, force x,y,z, torque x,y,z
start_state = state_recording[start_frame][0:6]
target_state = state_recording[target_frame][0:6]
transformation_gt_lin = -start_state  + target_state

S = kuka2dcm(start_state)
T = kuka2dcm(target_state)
S2T = np.linalg.inv(S) @ T
pose_gt = S2T

transformation_gt = S2T
print("start", start_state.round(3))
print("targt", target_state.round(3))
print()
print("lin", transformation_gt_lin.round(3))
print("gt ", dcm2pretty(S2T), "in TCP coordinates")

# Fitting Data

In [ ]:
from gym_grasping.flow_control.servoing_fitting import solve_transform
from scipy.spatial.transform import Rotation as R

# start
start_image = image_recording[start_frame]
start_depth = depth_recording[start_frame]
start_ee_pos = ee_positions[start_frame]
# target
end_image = image_recording[target_frame]
end_depth = depth_recording[target_frame]
end_mask = masks[target_frame]

# backward flow goes from (end_points -> start_points)
flow = servo_module.flow_module.step(end_image, start_image)
end_points = np.array(np.where(end_mask)).T
masked_flow = flow[end_mask]
start_points = end_points + masked_flow[:, ::-1].astype('int')

# Fitting without depth

In [ ]:
# Fitting code
end_points_hom = np.pad(end_points.astype('float'), ((0, 0), (0, 2)), mode="constant")
start_points_hom = np.pad(start_points.astype('float'), ((0, 0), (0, 2)), mode="constant")
#start_points_hom[:,0:4] = (T_tcp_cam @ start_points_hom[:,0:4].T).T
#end_points_hom[:,0:4] = (T_tcp_cam @ end_points_hom[:,0:4].T).T

# Trf. from camera(t+1) <- camera(t)
T_cp_c = solve_transform(start_points_hom, end_points_hom)

print("gt ", dcm2pretty(pose_gt))
print("img", dcm2pretty(T_cp_c))
print("img", dcm2pretty(T_tcp_cam @ T_cp_c @ np.linalg.inv(T_tcp_cam)), "in TCP coordinates")

# Fitting with depth

In [ ]:
C_X = 315.20367431640625
C_Y = 245.70614624023438
FOC_X = 617.8902587890625
FOC_Y = 617.8903198242188
print("T_tcp_cam euler angles", R.from_dcm(T_tcp_cam[:3,:3]).as_euler('xyz'))

def generate_pointcloud(rgb_image, depth_image, masked_points):
    pointcloud = []
    for u, v in masked_points:
        try:
            Z = depth_image[u, v] * 0.000125
            color = rgb_image[u, v]
        except IndexError:
            Z = 0
            color = 0,0,0
        X = (v - C_X) * Z / FOC_X
        Y = (u - C_Y) * Z / FOC_Y
        pointcloud.append([X, Y, Z, 1, *color])
    pointcloud = np.array(pointcloud)
    return pointcloud

K = np.array([[617.89, 0, 315.2, 0 ],
              [0, 617.89, 245.7, 0 ],
              [0, 0, 1, 0]])

def project(K, X):
    x = K @ X
    return x[0:2] / x[2]

start_pc = generate_pointcloud(start_image, start_depth, start_points)
end_pc = generate_pointcloud(end_image, end_depth, end_points)

mask_pc = np.logical_and( start_pc[:,2]!=0 , end_pc[:,2] !=0 )
mask_pc = np.logical_and( mask_pc, np.random.random(mask_pc.shape[0])>.95) 

start_pc = start_pc[mask_pc]
end_pc = end_pc[mask_pc]

# transform into TCP coordinates
start_pc[:,0:4] = (T_tcp_cam @ start_pc[:,0:4].T).T
end_pc[:,0:4] = (T_tcp_cam @ end_pc[:,0:4].T).T

if False:
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.set_xlim3d(-.15, 0.15)
    ax.set_ylim3d(-.15, 0.15)
    ax.set_zlim3d(0, 0.3)
    ax.scatter(np.reshape(start_pc[:,0], -1), np.reshape(start_pc[:,1], -1),np.reshape(start_pc[:,2], -1), c=start_pc[:,3:6]/255)
    ax.scatter(np.reshape(end_pc[:,0], -1), np.reshape(end_pc[:,1], -1),np.reshape(end_pc[:,2], -1), c=end_pc[:,3:6][:,::-1]/255)
    for i in np.linspace(0,.1,10):
        ax.scatter(0,0,-i)
    plt.show()
    
T_tp_t = solve_transform(start_pc[:,0:4], end_pc[:,0:4])

print("gt ", dcm2pretty(pose_gt))
print("dpt", dcm2pretty(np.linalg.inv(T_tp_t)))

print("guess  rot", dcm2pretty(np.linalg.inv(T_tp_t))[5])
print("action rot", actions[start_frame][3])

# Center of Rotation from Image
This code finds the center of rotation in image plane.

In [ ]:
flow_img = servo_module.flow_module.computeImg(flow, dynamic_range=False)
flow_norm = np.linalg.norm(flow, axis=2)
# show segmentatione edge

h, w, _ = flow.shape
ch, cw = h/2, w/2

from skimage.feature import peak_local_max
d = 50
subsection = flow_norm[int(ch)-d:int(ch)+d,int(cw)-d:int(cw)+d]
peak_h,peak_w = peak_local_max(-subsection)[0]
dh, dw = peak_h - d, peak_w - d

# show loss, frame number
fig, ax = plt.subplots(1)
ax.imshow(flow_img)
ax.scatter(cw, ch, s=25, c='red', marker='x')
ax.scatter(cw+dw, ch+dh, s=25, c='green', marker='x')
plt.show()

#ax.imshow(subsection)
#ax.scatter(dw,dh,s=25,c='red', marker='x')
print("delta height, width:", dh,dw)
